In [1]:
from keras.applications.resnet_v2 import ResNet50V2 , preprocess_input as resnet_preprocess
from keras.applications.densenet import DenseNet121, preprocess_input as densenet_preprocess
from keras.layers import concatenate
from keras.models import Model
from keras.layers import Input, Lambda
input_shape = (331,331,3)
input_layer = Input(shape=input_shape)#first feature extractor
preprocessor_resnet = Lambda(resnet_preprocess)(input_layer)
resnet50v2 = ResNet50V2(weights = 'imagenet',include_top = False,input_shape = input_shape,pooling ='avg')(preprocessor_resnet)
preprocessor_densenet = Lambda(densenet_preprocess)(input_layer)
densenet = DenseNet121(weights = 'imagenet',include_top = False,input_shape = input_shape,pooling ='avg')(preprocessor_densenet)
merge = concatenate([resnet50v2,densenet])
stacked_model = Model(inputs = input_layer, outputs = merge)
stacked_model.summary()

29084464/29084464 [==============================] - 6s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 331, 331, 3)]        0         []                            
                                                                                                  
 lambda (Lambda)             (None, 331, 331, 3)          0         ['input_1[0][0]']             
                                                                                                  
 lambda_1 (Lambda)           (None, 331, 331, 3)          0         ['input_1[0][0]']             
                                                                                                  
 resnet50v2 (Functional)     (None, 2048)                 2356480   ['lambda[0][0]']              
                             

In [2]:
import os
import shutil
import pandas as pd
train_dir = 'C:\\Users\\abhir\\Downloads\\dog-breed-identification\\train\\'
labels_dataframe = pd.read_csv('C:\\Users\\abhir\\Downloads\\dog-breed-identification\\labels.csv')
dog_breeds = sorted(list(set(labels_dataframe['breed'])))
n_classes = len(dog_breeds)
class_to_num = dict(zip(dog_breeds, range(n_classes)))
labels_dataframe['file_path'] = labels_dataframe['id'].apply(lambda x:train_dir+f"{x}.jpg")
labels_dataframe['breed'] = labels_dataframe.breed.map(class_to_num)

In [3]:
from keras.utils import to_categorical
y_train = to_categorical(labels_dataframe.breed)

In [4]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tqdm import tqdm

def feature_extractor(df):
    img_size = (331, 331, 3)
    data_size = len(df)
    batch_size = 20
    X = np.zeros([data_size, 3072], dtype=np.uint8)

    
    datagen = ImageDataGenerator()

    generator = datagen.flow_from_dataframe(df,
        x_col='file_path', class_mode=None,batch_size=20, shuffle=False, target_size=(img_size[:2]), color_mode='rgb')
    i = 0

    for input_batch in tqdm(generator):
        input_batch = stacked_model.predict(input_batch)
        X[i * batch_size: (i + 1) * batch_size] = input_batch
        i += 1
        if i * batch_size >= data_size:
            break
    return X

X_train = feature_extractor(labels_dataframe)


Found 10222 validated image filenames.


  0%|          | 0/512 [00:00<?, ?it/s]

1/1 [==============================] - 6s 6s/step


  0%|          | 1/512 [00:06<54:56,  6.45s/it]

1/1 [==============================] - 4s 4s/step


...

100%|█████████▉| 510/512 [37:40<00:09,  4.52s/it]

1/1 [==============================] - 4s 4s/step


100%|█████████▉| 511/512 [37:44<00:04,  4.46s/it]

1/1 [==============================] - 1s 522ms/step


100%|█████████▉| 511/512 [37:44<00:04,  4.43s/it]


In [7]:
import keras
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout
predictor_model = keras.models.Sequential([
    InputLayer(X_train.shape[1:]),
    Dropout(0.7),
    Dense(n_classes, activation='softmax')
])
predictor_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
#Prepare call backs
EarlyStop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('/kaggle/working/checkpoing',
                             monitor = 'val_loss',mode = 'min',save_best_only= True)
lr = ReduceLROnPlateau(monitor = 'val_loss',factor = 0.5,patience = 3,min_lr = 0.00001)
my_callback=[EarlyStop_callback,checkpoint]

In [8]:
#Train simple DNN on extracted features.# here X is bottleneck feature extracted by using stacked pretrained model 
history_graph = predictor_model.fit(X_train , y_train,
            batch_size=128,
            epochs=60,
            validation_split=0.1 ,
           callbacks = my_callback)

Epoch 1/60
70/72 [============================>.] - ETA: 0s - loss: 3.3195 - accuracy: 0.2735INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


72/72 [==============================] - 1s 15ms/step - loss: 3.2888 - accuracy: 0.2787 - val_loss: 1.4927 - val_accuracy: 0.7840
Epoch 2/60
69/72 [===========================>..] - ETA: 0s - loss: 1.3529 - accuracy: 0.6810INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


72/72 [==============================] - 1s 12ms/step - loss: 1.3417 - accuracy: 0.6829 - val_loss: 0.8626 - val_accuracy: 0.8397
Epoch 3/60
69/72 [===========================>..] - ETA: 0s - loss: 0.8865 - accuracy: 0.7884INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


...

72/72 [==============================] - 1s 15ms/step - loss: 0.2528 - accuracy: 0.9373 - val_loss: 0.4524 - val_accuracy: 0.8749
Epoch 15/60
72/72 [==============================] - 1s 10ms/step - loss: 0.2478 - accuracy: 0.9371 - val_loss: 0.4540 - val_accuracy: 0.8641
Epoch 16/60
72/72 [==============================] - 1s 11ms/step - loss: 0.2387 - accuracy: 0.9399 - val_loss: 0.4555 - val_accuracy: 0.8661
Epoch 17/60
72/72 [==============================] - 1s 11ms/step - loss: 0.2233 - accuracy: 0.9448 - val_loss: 0.4573 - val_accuracy: 0.8651
Epoch 18/60
69/72 [===========================>..] - ETA: 0s - loss: 0.2110 - accuracy: 0.9464INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


72/72 [==============================] - 1s 20ms/step - loss: 0.2118 - accuracy: 0.9464 - val_loss: 0.4499 - val_accuracy: 0.8651
Epoch 19/60
72/72 [==============================] - 1s 11ms/step - loss: 0.2021 - accuracy: 0.9481 - val_loss: 0.4521 - val_accuracy: 0.8622
Epoch 20/60
72/72 [==============================] - 1s 10ms/step - loss: 0.1882 - accuracy: 0.9529 - val_loss: 0.4516 - val_accuracy: 0.8622
Epoch 21/60
71/72 [============================>.] - ETA: 0s - loss: 0.1846 - accuracy: 0.9516INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


INFO:tensorflow:Assets written to: /kaggle/working\checkpoing\assets


72/72 [==============================] - 1s 15ms/step - loss: 0.1847 - accuracy: 0.9516 - val_loss: 0.4493 - val_accuracy: 0.8671
Epoch 22/60
72/72 [==============================] - 1s 11ms/step - loss: 0.1810 - accuracy: 0.9549 - val_loss: 0.4504 - val_accuracy: 0.8700
Epoch 23/60
72/72 [==============================] - 1s 12ms/step - loss: 0.1726 - accuracy: 0.9591 - val_loss: 0.4525 - val_accuracy: 0.8651
Epoch 24/60
72/72 [==============================] - 1s 12ms/step - loss: 0.1659 - accuracy: 0.9605 - val_loss: 0.4529 - val_accuracy: 0.8631
Epoch 25/60
72/72 [==============================] - 1s 13ms/step - loss: 0.1635 - accuracy: 0.9572 - val_loss: 0.4523 - val_accuracy: 0.8592
Epoch 26/60
72/72 [==============================] - 1s 11ms/step - loss: 0.1574 - accuracy: 0.9599 - val_loss: 0.4585 - val_accuracy: 0.8524
Epoch 27/60
72/72 [==============================] - 1s 11ms/step - loss: 0.1642 - accuracy: 0.9573 - val_loss: 0.4563 - val_accuracy: 0.8543
Epoch 28/60
72/72 

In [11]:
predictor_model.save('C:\\Users\\abhir\\Downloads\\dogbreed.h5')
stacked_model.save('C:\\Users\\abhir\\Downloads\\feature_extractor.h5')

In [14]:
from keras.preprocessing.image import load_img, img_to_array
img = load_img("C:\\Users\\abhir\\Downloads\\lebra.jpg", target_size=(331,331))
img = img_to_array(img)
img = np.expand_dims(img,axis = 0) # this is creating tensor(4Dimension)
    
extracted_features = stacked_model.predict(img)
y_pred = predictor_model.predict(extracted_features)

1/1 [==============================] - 0s 31ms/step


In [16]:
def get_key(val): 
        for key, value in class_to_num.items(): 
         if val == value: 
             return key 
            
pred_codes = np.argmax(y_pred, axis = 1)
predicted_dog_breed = get_key(pred_codes)


In [17]:
print(predicted_dog_breed)

labrador_retriever
